<a href="https://colab.research.google.com/github/cantalk-bolt/crypto-trading-bot/blob/main/bist_veri_cekme_ipynb_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
# Gerekli kütüphaneleri yükle
!pip install git+https://github.com/rongardF/tvdatafeed --quiet

import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
import warnings

warnings.simplefilter(action='ignore')

# Basit Hareketli Ortalama
def sma(series, length):
    return series.rolling(window=length).mean()

# Üssel Hareketli Ortalama
def ema(series, length):
    return series.ewm(span=length, adjust=False).mean()

# Bankery Stratejisi
def Bankery(data):
    df = data.copy()
    close_minus_rolling_min = df['close'] - df['low'].rolling(window=27).min()
    percentage_change = close_minus_rolling_min / (df['high'].rolling(window=27).max() - df['low'].rolling(window=27).min()) * 100

    sma1 = sma(percentage_change, 5)
    sma2 = sma(sma1, 3)

    fundtrend = (3 * sma1 - 2 * sma2 - 50) * 1.032 + 50

    typ = (2 * df['close'] + df['high'] + df['low'] + df['open']) / 5
    lol = df['low'].rolling(window=34).min()
    hoh = df['high'].rolling(window=34).max()
    bullbearline = ema((typ - lol) / (hoh - lol) * 100, 13)

    bankerentry = (fundtrend > bullbearline) & (bullbearline < 25)
    df['Entry'] = bankerentry.fillna(False)
    return df

# TV login (anonim de çalışır)
tv = TvDatafeed()

# Manuel olarak BIST hisseleri listesi (örnek olarak BIST30 hisseleri girildi)
Hisseler = [
    'A1CAP', 'ACSEL', 'ADEL', 'ADESE', 'ADGYO', 'AEFES', 'AFYON', 'AGESA', 'AGHOL', 'AGROT',
    'AGYO', 'AHGAZ', 'AHSGY', 'AKBNK', 'AKCNS', 'AKENR', 'AKFGY', 'AKGRT', 'AKMGY', 'AKSA',
    'AKSEN', 'AKSGY', 'AKYHO', 'ALARK', 'ALBRK', 'ALCAR', 'ALCTL', 'ALGYO', 'ALKA', 'ALKIM',
    'ALMAD', 'ALNTF', 'ALYAG', 'ANACM', 'ANELE', 'ANGEN', 'ANHYT', 'ANSGR', 'ARASE', 'ARCLK',
    'ARDYZ', 'ARENA', 'ARMDA', 'ARSAN', 'ARTMS', 'ARZUM', 'ASELS', 'ASGYO', 'ASUZU', 'ATAGY',
    'ATATP', 'ATEKS', 'ATLAS', 'ATSYH', 'AVHOL', 'AVGYO', 'AVOD', 'AVTUR', 'AYCES', 'AYDEM',
    'AYEN', 'AYES', 'AYGAZ', 'AZTEK', 'BAGFS', 'BAKAB', 'BALAT', 'BANVT', 'BARMA', 'BASCM',
    'BASGZ', 'BAYRK', 'BERA', 'BERK', 'BEYAZ', 'BFREN', 'BGCYO', 'BIMAS', 'BIOEN', 'BIZIM',
    'BJKAS', 'BLCYT', 'BLGYO', 'BMSCH', 'BMSTL', 'BNTAS', 'BOBET', 'BOSSA', 'BRISA', 'BRKSN',
    'BRLSM', 'BRMEN', 'BRYAT', 'BSOKE', 'BTCIM', 'BUCIM', 'BUDAS', 'BURCE', 'BURVA', 'CANTE',
    'CARFA', 'CASA', 'CCOLA', 'CELHA', 'CEMAS', 'CEMTS', 'CIMSA', 'CMENT', 'CMBTN', 'COSMO',
    'CRDFA', 'CRFSA', 'CUSAN', 'CWENE', 'DAPGM', 'DARDL', 'DENGE', 'DERHL', 'DERIM', 'DESA',
    'DEVA', 'DGATE', 'DGGYO', 'DGKLB', 'DGNMO', 'DGNYO', 'DGRTR', 'DGSER', 'DGUY', 'DHA',
    'DIRIT', 'DITAS', 'DNISI', 'DOAS', 'DOBUR', 'DOGUB', 'DOHOL', 'DOMINO', 'DURDO', 'DYOBY',
    'DZGYO', 'ECILC', 'ECZYT', 'EDATA', 'EDIP', 'EGEEN', 'EGGUB', 'EGPRO', 'EGSER', 'EİS',
    'EKGYO', 'EKIZ', 'ELITE', 'ELMAS', 'EMKEL', 'EMNIS', 'ENKAI', 'ENJSA', 'ENSRI', 'EPLAS',
    'ERBOS', 'EREGL', 'ERSU', 'ESCAR', 'ESCOM', 'ESEN', 'ESKOM', 'ETILR', 'EUHOL', 'EUKYO',
    'EYGYO', 'FADE', 'FENER', 'FENIS', 'FLAP', 'FLMNC', 'FMIZP', 'FONET', 'FORMT', 'FORTE',
    'FRIGO', 'FROTO', 'FZLGY', 'GARAN', 'GATEK', 'GEDIK', 'GEDZA', 'GENTS', 'GEREL', 'GESAN',
    'GIPTA', 'GLBMD', 'GLRYH', 'GLYHO', 'GMAKT', 'GOLTS', 'GOODY', 'GOZDE', 'GRNYO', 'GSDDE',
    'GSDHO', 'GSRAY', 'GUBRF', 'GUHEM', 'GUSGR', 'GWIND', 'HALKB', 'HATEK', 'HDFGS', 'HEDEF',
    'HEKTS', 'HGGYO', 'HKTM', 'HLGYO', 'HUBVC', 'HUNER', 'HURC', 'ICBCT', 'IDEAS', 'IDGYO',
    'IDTUR', 'IEYHO', 'IHEVA', 'IHLAS', 'IHLGM', 'IHLAS', 'ISCTR', 'ISGYO', 'ISMEN', 'IZFAS', 'IZINV', 'IZMDC', 'IZTAR', 'KAPLM', 'KARSN', 'KATMR',
    'KCHOL', 'KERVT', 'KFEIN', 'KIMMR', 'KLGYO', 'KLKIM', 'KMPUR', 'KONTR', 'KORDS', 'KOZAA',
    'KOZAL', 'KRDMA', 'KRDMB', 'KRDMD', 'KRONT', 'KRSTL', 'KRVGD', 'KSTUR', 'KUTPO', 'LIDFA',
    'LINK', 'LKMNH', 'LOGO', 'LUKSK', 'MAALT', 'MAGEN', 'MAKIM', 'MARTI', 'MAVI', 'MBAS',
    'MEGAP', 'MERCN', 'MERIT', 'METRO', 'METUR', 'MIPAZ', 'MNDTR', 'MNDRS', 'MOBTL', 'MPARK',
    'MRDIN', 'MRSHL', 'MSGYO', 'MTRYO', 'NETAS', 'NIBAS', 'NUGYO', 'NUHCM', 'ODAS', 'OFGYO',
    'OHEVA', 'OLMIP', 'ORCAY', 'ORGE', 'ORMA', 'OSTIM', 'OTKAR', 'OYAKC', 'OYLUM', 'OZBAL',
    'OZGYO', 'OZKGY', 'PAGYO', 'PAMEL', 'PAMSU', 'PARSN', 'PASEU', 'PENTA', 'PENGD', 'PETKM',
    'PGSUS', 'PKART', 'PKENT', 'PLTUR', 'PRKAB', 'PRKME', 'PRZMA', 'PSGYO', 'QNBFB', 'QUAGR',
    'RALYH', 'RAYSG', 'REEDR', 'RODRG', 'ROYAL', 'RUBNS', 'RYSAS', 'RYGYO', 'SANEL', 'SANKO',
    'SASA', 'SAYAS', 'SEKUR', 'SELEC', 'SELGD', 'SEYKM', 'SILVR', 'SIMEK', 'SINMA', 'SKBNK',
    'SKTAS', 'SMART', 'SMRTG', 'SNGYO', 'SNPAM', 'SODA', 'SOKE', 'SOKM', 'SONME', 'SPEAS',
    'SRVGY', 'SUMAS', 'SUNTK', 'SUWEN', 'TATGD', 'TAVHL', 'TBORG', 'TCELL', 'TCHOL', 'TEKTU',
    'TEZOL', 'THYAO', 'TKFEN', 'TKNSA', 'TLMAN', 'TOASO', 'TRCAS', 'TRGYO', 'TRILC', 'TRKCM',
    'TSPOR', 'TTRAK', 'TUKAS', 'TUPRS', 'TURSG', 'UFUK', 'ULKER', 'VERUS', 'VESTL', 'VAKBN', 'VAKFN', 'VAKKO', 'VANGD', 'VBTYZ', 'VERTU', 'VERUS', 'VESBE', 'VESTL', 'VKFYO', 'VKGYO', 'VKING', 'VRGYO', 'YAPRK', 'YATAS', 'YAYLA', 'YBTAS', 'YEOTK', 'YESIL', 'YGGYO', 'YGYO', 'YIGIT', 'YKBNK', 'YKSLN', 'YONGA', 'YUNSA', 'YYAPI', 'YYLGD'
]

# Raporlama DataFrame
Titles = ['Hisse Adı', 'Son Fiyat', 'Dip Sinyali']
df_signals = pd.DataFrame(columns=Titles)

# Tüm hisselerde Bankery kontrolü
for hisse in Hisseler:
    try:
        data = tv.get_hist(symbol=hisse, exchange='BIST', interval=Interval.in_4_hour, n_bars=100)
        if data is None or data.empty:
            continue

        data = data.reset_index()
        Banker = Bankery(data)
        Banker.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
        Banker.set_index('datetime', inplace=True)

        Signals = Banker.tail(2).reset_index()
        Entry = (Signals.loc[0, 'Entry'] == False) and (Signals.loc[1, 'Entry'] == True)
        Last_Price = Signals.loc[1, 'Close']

        df_signals.loc[len(df_signals)] = [hisse, Last_Price, Entry]
        print(f"{hisse} - Son Fiyat: {Last_Price:.2f} - Dip Sinyali: {Entry}")

    except Exception as e:
        print(f"Hata oluştu: {hisse} - {e}")
        continue

# Sadece sinyal gelenleri göster
df_True = df_signals[df_signals['Dip Sinyali'] == True]
print("\n--- Dip Sinyali Gelen Hisseler ---")
print(df_True)


  Preparing metadata (setup.py) ... done


A1CAP - Son Fiyat: 5.08 - Dip Sinyali: False
ACSEL - Son Fiyat: 108.60 - Dip Sinyali: False
ADEL - Son Fiyat: 31.88 - Dip Sinyali: True
ADESE - Son Fiyat: 2.61 - Dip Sinyali: False
ADGYO - Son Fiyat: 28.94 - Dip Sinyali: False
AEFES - Son Fiyat: 151.30 - Dip Sinyali: False
AFYON - Son Fiyat: 13.41 - Dip Sinyali: False
AGESA - Son Fiyat: 141.80 - Dip Sinyali: False
AGHOL - Son Fiyat: 276.25 - Dip Sinyali: False
AGROT - Son Fiyat: 8.26 - Dip Sinyali: False
AGYO - Son Fiyat: 6.20 - Dip Sinyali: False
AHGAZ - Son Fiyat: 22.10 - Dip Sinyali: False
AHSGY - Son Fiyat: 20.88 - Dip Sinyali: True
AKBNK - Son Fiyat: 49.28 - Dip Sinyali: False
AKCNS - Son Fiyat: 148.70 - Dip Sinyali: False
AKENR - Son Fiyat: 12.68 - Dip Sinyali: False
AKFGY - Son Fiyat: 2.17 - Dip Sinyali: False
AKGRT - Son Fiyat: 6.12 - Dip Sinyali: False
AKMGY - Son Fiyat: 200.00 - Dip Sinyali: False
AKSA - Son Fiyat: 9.42 - Dip Sinyali: False
AKSEN - Son Fiyat: 31.82 - Dip Sinyali: False
AKSGY - Son Fiyat: 6.23 - Dip Sinyali: F

ERROR:tvDatafeed.main:Connection timed out
ERROR:tvDatafeed.main:no data, please check the exchange and symbol
ERROR:tvDatafeed.main:Connection timed out
ERROR:tvDatafeed.main:no data, please check the exchange and symbol
ERROR:tvDatafeed.main:Connection timed out
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


ANELE - Son Fiyat: 15.76 - Dip Sinyali: False
ANGEN - Son Fiyat: 10.95 - Dip Sinyali: False
ANHYT - Son Fiyat: 78.60 - Dip Sinyali: False
ANSGR - Son Fiyat: 90.10 - Dip Sinyali: False
ARASE - Son Fiyat: 43.40 - Dip Sinyali: False
ARCLK - Son Fiyat: 116.00 - Dip Sinyali: False
ARDYZ - Son Fiyat: 25.66 - Dip Sinyali: False
ARENA - Son Fiyat: 46.22 - Dip Sinyali: False


ERROR:tvDatafeed.main:Connection timed out
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


...

SRVGY - Son Fiyat: 3.11 - Dip Sinyali: False
SUMAS - Son Fiyat: 330.00 - Dip Sinyali: False
SUNTK - Son Fiyat: 40.32 - Dip Sinyali: False
SUWEN - Son Fiyat: 12.88 - Dip Sinyali: False
TATGD - Son Fiyat: 11.74 - Dip Sinyali: False
TAVHL - Son Fiyat: 229.10 - Dip Sinyali: False
TBORG - Son Fiyat: 173.00 - Dip Sinyali: False
TCELL - Son Fiyat: 92.65 - Dip Sinyali: False


ERROR:tvDatafeed.main:Connection timed out
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


TEKTU - Son Fiyat: 4.41 - Dip Sinyali: False
TEZOL - Son Fiyat: 17.18 - Dip Sinyali: False
THYAO - Son Fiyat: 291.00 - Dip Sinyali: False
TKFEN - Son Fiyat: 125.50 - Dip Sinyali: False
TKNSA - Son Fiyat: 23.74 - Dip Sinyali: False
TLMAN - Son Fiyat: 78.55 - Dip Sinyali: False
TOASO - Son Fiyat: 200.90 - Dip Sinyali: False
TRCAS - Son Fiyat: 27.82 - Dip Sinyali: False
TRGYO - Son Fiyat: 60.00 - Dip Sinyali: False
TRILC - Son Fiyat: 19.34 - Dip Sinyali: False


ERROR:tvDatafeed.main:Connection timed out
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


TSPOR - Son Fiyat: 0.90 - Dip Sinyali: False
TTRAK - Son Fiyat: 587.00 - Dip Sinyali: False
TUKAS - Son Fiyat: 2.65 - Dip Sinyali: False
TUPRS - Son Fiyat: 131.40 - Dip Sinyali: False
TURSG - Son Fiyat: 17.18 - Dip Sinyali: False
UFUK - Son Fiyat: 855.00 - Dip Sinyali: False
ULKER - Son Fiyat: 112.40 - Dip Sinyali: False
VERUS - Son Fiyat: 240.00 - Dip Sinyali: False
VESTL - Son Fiyat: 41.20 - Dip Sinyali: False
VAKBN - Son Fiyat: 22.26 - Dip Sinyali: False
VAKFN - Son Fiyat: 2.34 - Dip Sinyali: False
VAKKO - Son Fiyat: 63.45 - Dip Sinyali: False
VANGD - Son Fiyat: 33.96 - Dip Sinyali: False
VBTYZ - Son Fiyat: 18.33 - Dip Sinyali: False
VERTU - Son Fiyat: 32.92 - Dip Sinyali: False
VERUS - Son Fiyat: 240.00 - Dip Sinyali: False
VESBE - Son Fiyat: 11.50 - Dip Sinyali: False
VESTL - Son Fiyat: 41.18 - Dip Sinyali: False
VKFYO - Son Fiyat: 17.25 - Dip Sinyali: True
VKGYO - Son Fiyat: 1.97 - Dip Sinyali: False
VKING - Son Fiyat: 27.30 - Dip Sinyali: False
VRGYO - Son Fiyat: 2.49 - Dip Siny

In [ ]:
!pip install tradingview-ta
from tradingview_ta import TA_Handler, Interval, Exchange
# ... (rest of the code) ...

tv = TvDatafeed()
handler = TA_Handler(
    symbol="US500",
    screener="america",
    exchange="CME",
    interval=Interval.INTERVAL_1_HOUR
)

Hisseler = handler.get_analysis().summary
# Assuming 'Hisseler' is a list of symbols, proceed with your analysis

# ... (rest of your code) ...

Exception: Exchange or symbol not found.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Load a prebuilt dataset.
1. Build a neural network machine learning model that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.

This tutorial is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. To run all the code in the notebook, select **Runtime** > **Run all**. To run the code cells one at a time, hover over each cell and select the **Run cell** icon.

![Run cell icon](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/images/beginner/run_cell_icon.png?raw=1)

## Set up TensorFlow

Import TensorFlow into your program to get started:

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

If you are following along in your own development environment, rather than [Colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb), see the [install guide](https://www.tensorflow.org/install) for setting up TensorFlow for development.

Note: Make sure you have upgraded to the latest `pip` to install the TensorFlow 2 package if you are using your own development environment. See the [install guide](https://www.tensorflow.org/install) for details.

## Load a dataset

Load and prepare the MNIST dataset. The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by `255.0`. This also converts the sample data from integers to floating-point numbers:

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Build a machine learning model

Build a `tf.keras.Sequential` model:

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

[`Sequential`](https://www.tensorflow.org/guide/keras/sequential_model) is useful for stacking layers where each layer has one input [tensor](https://www.tensorflow.org/guide/tensor) and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), and [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers.

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [ ]:
predictions = model(x_train[:1]).numpy()
predictions

The `tf.nn.softmax` function converts these logits to *probabilities* for each class:

In [ ]:
tf.nn.softmax(predictions).numpy()

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

Define a loss function for training using `losses.SparseCategoricalCrossentropy`:

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [ ]:
loss_fn(y_train[:1], predictions).numpy()

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss:

In [ ]:
model.fit(x_train, y_train, epochs=5)

The `Model.evaluate` method checks the model's performance, usually on a [validation set](https://developers.google.com/machine-learning/glossary#validation-set) or [test set](https://developers.google.com/machine-learning/glossary#test-set).

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
probability_model(x_test[:5])

## Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.

For more examples of using Keras, check out the [tutorials](https://www.tensorflow.org/tutorials/keras/). To learn more about building models with Keras, read the [guides](https://www.tensorflow.org/guide/keras). If you want learn more about loading and preparing data, see the tutorials on [image data loading](https://www.tensorflow.org/tutorials/load_data/images) or [CSV data loading](https://www.tensorflow.org/tutorials/load_data/csv).
